<a href="https://colab.research.google.com/github/JakeTurner616/Adonalsium/blob/LLM/trainandgenerate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install dependencies

In [6]:
! pip install torch pandas
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 16.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


Download the book

In [2]:
import requests

# URLs for the data
book_url = 'https://gist.githubusercontent.com/JakeTurner616/7c5bcb37d861ba6fdb66d12a9bb9a084/raw/63d14a3aeed74a23b7aa1dba263b42d7c2b8676e/book.txt'  # Change to your book's URL

# Download the book
book_response = requests.get(book_url)
book_text = book_response.text

# Save the book text to a file (optional, you can also directly use `book_text` for training)
with open('reverted_book.txt', 'w') as f:
    f.write(book_text)

print("Downloaded and saved the book text.")

Downloaded and saved the book text.


Train the model

In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set pad token to eos token for GPT-2 compatibility

# Prepare dataset
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="reverted_book.txt",
    block_size=128
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

model = GPT2LMHeadModel.from_pretrained('gpt2')

training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,3.298500
1000,3.139500
1500,3.091500
2000,3.051400
2500,2.997500
3000,2.998600
3500,2.946400
4000,2.941900
4500,2.946700
5000,2.947700


TrainOutput(global_step=43482, training_loss=2.736691640856777, metrics={'train_runtime': 7676.1434, 'train_samples_per_second': 22.657, 'train_steps_per_second': 5.665, 'total_flos': 1.1360912228352e+16, 'train_loss': 2.736691640856777, 'epoch': 3.0})

Generate text

In [35]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, pipeline

def generate_text(prompt, model_path='./results/checkpoint-40000/'):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token

    text_generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
    generated_texts = text_generator(
        prompt,
        max_length=150,
        temperature=0.7,
        top_p=0.85,
        repetition_penalty=1.3,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True
      )

    for generated in generated_texts:
        print(generated['generated_text'])

if __name__ == "__main__":
    prompt = "She fell to the ground dead"
    generate_text(prompt)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


She fell to the ground dead, and it was a miracle she survived. Her hands were covered in blood, her feet scratched by something sharp.
  Mistings are supposed not as dangerous as you think, Vin thought. She pulled back against the wall beside Elend. He had fallen unconscious there too. How did he survive? What if his wounds weren’t so bad that they made him sick? It didn't matter—he would die soon anyway. The only way out of this place was for them all together into one....
 Something crashed through Kelsier on top of Yomen. A man with dark brown skin. Like Ruin himself, but different. Marsh looked like someone else from what could be seen up above
